In [1]:
from __future__ import print_function

import numpy as np
import scipy.misc
import scipy.io
import scipy as sc
import tensorflow as tf
from PIL import Image

from scipy.optimize import fmin_cg

from keras.layers import Input
from keras.applications import vgg19
from keras.optimizers import Adam
from keras.models import Model

import keras.backend as K

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

Using TensorFlow backend.


In [2]:
class CONFIG:
    IMAGE_WIDTH = 400
    IMAGE_HEIGHT = 300
    COLOR_CHANNELS = 3
    CONTENT_WEIGHT = 5
    STYLE_WEIGHT = 100
    TOTAL_VARIATION_WEIGHT = 1.
    NOISE_RATIO = .6
    MEANS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))

In [3]:
IMAGE_SHAPE = (1,
                CONFIG.IMAGE_HEIGHT,
                CONFIG.IMAGE_WIDTH,
                CONFIG.COLOR_CHANNELS)

In [4]:
STYLE_LAYERS = [
    ('block1_conv1', .5),
#     ('block1_conv2', .2),
    ('block2_conv1', .5),
#     ('block2_conv2', .2),
    ('block3_conv1', .5),
#     ('block3_conv2', 0.2),
#     ('block3_conv3', .2),
#     ('block3_conv4', 0.2),
    ('block4_conv1', .5),
#     ('block4_conv2', 0.2),
#     ('block4_conv3', 0.2),
#     ('block4_conv4', .2),
    ('block5_conv1', .5),
#     ('block5_conv2', 0.2),
#     ('block5_conv3', 0.2),
#     ('block5_conv4', .2)
]

In [5]:
def reshape_and_normalize_image(image_path):
    """
    Reshape and normalize the input image (content or style)
    """
    
    image = Image.open(image_path)
    image = image.resize((CONFIG.IMAGE_WIDTH, CONFIG.IMAGE_HEIGHT))
    
    image_array = np.asarray(image, dtype='float32')
    image = np.expand_dims(image_array, axis=0)
    
    # Substract the mean to match the expected input of VGG16
    image = image - CONFIG.MEANS
    
    return image

In [6]:
def compute_content_loss(content, generated):
    n_H, n_W, n_C = generated.get_shape().as_list()

    content = K.reshape(content, [-1])
    generated = K.reshape(generated, [-1])

    return K.sum(K.square(content - generated)) / (4*n_H*n_W*n_C)

In [7]:
def gram_matrix(x):
    gram = K.dot(x, K.transpose(x))
    return gram

def style_layer_loss(style, generated):
    n_H, n_W, n_C = generated.get_shape().as_list()
    
    style = K.reshape(style, (n_C, n_H*n_W))
    generated = K.reshape(generated, (n_C, n_H*n_W))
    
    S = gram_matrix(style)
    G = gram_matrix(generated)

    channels = n_C
    size = n_H*n_W

    return K.sum(K.square(S - G)) / (4. * (channels ** 2) * (size ** 2))

In [8]:
def save_image(path, image):
    
    # Un-normalize the image so that it looks good
    image = image + CONFIG.MEANS
    
    # Clip and Save the image
    image = np.clip(image[0], 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)

In [9]:
sess = tf.Session()
K.set_session(sess)

In [10]:
content_image_path = "./images/louvre_small.jpg"
content_image = K.variable(reshape_and_normalize_image(content_image_path))

style_image_path = "./images/monet.jpg"
style_image = K.variable(reshape_and_normalize_image(style_image_path))

generated_image = K.placeholder(IMAGE_SHAPE)

In [11]:
# combine the 3 images into a single Keras tensor
input_image = K.concatenate([content_image,
                            style_image,
                            generated_image], axis=0)

In [12]:
model = vgg19.VGG19(weights=None, input_tensor=input_image, include_top=False)
model.load_weights("./vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [13]:
# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [14]:
# extract features only from the content layer
features = outputs_dict["block4_conv2"]

In [15]:
content_features = features[0,:,:,:]
generated_features = features[2,:,:,:]

content_loss = compute_content_loss(content_features, generated_features)

In [16]:
style_loss = K.variable(0.0)

for layer_name, coeff in STYLE_LAYERS:
    features = outputs_dict[layer_name]
    style_features = features[1,:,:,:]
    generated_features = features[2,:,:,:]
    style_loss += coeff * style_layer_loss(style_features, generated_features)

In [17]:
total_loss = content_loss + style_loss

In [18]:
# compute gradients of output img with respect to loss
grads = K.gradients(total_loss, generated_image)

In [19]:
outputs = [total_loss]
outputs += grads
loss_grads_function = K.function([generated_image], outputs)

In [44]:
class Evaluator:

    def calculate_loss_grads(self, x):
        self.out = loss_grads_function([x])

    def get_loss(self, x):
        x = x.reshape(IMAGE_SHAPE)
        self.out = loss_grads_function([x])
        return self.out[0]

    def get_grads(self, x):
        x = x.reshape(IMAGE_SHAPE)
        self.out = loss_grads_function([x])
        print(self.out[1].shape)
        return np.array(self.out[1]).flatten()

In [45]:
x = np.random.uniform(0, 255, IMAGE_SHAPE) - 128.

In [46]:
evaluator = Evaluator()
evaluator.calculate_loss_grads(x)

In [ ]:
y, min_val, info = fmin_cg(evaluator.get_loss,
                           x.flatten(),
                           fprime=evaluator.get_grads,
                           maxiter=10)

(1, 300, 400, 3)
(1, 300, 400, 3)
